In [2]:
robot_code = [3,8,1005,8,291,1106,0,11,0,0,0,104,1,104,0,3,8,1002,8,-1,10,101,1,10,10,4,10,108,0,8,10,4,10,1002,8,1,28,1,1003,20,10,2,1103,19,10,3,8,1002,8,-1,10,1001,10,1,10,4,10,1008,8,0,10,4,10,1001,8,0,59,1,1004,3,10,3,8,102,-1,8,10,1001,10,1,10,4,10,108,0,8,10,4,10,1001,8,0,84,1006,0,3,1,1102,12,10,3,8,1002,8,-1,10,101,1,10,10,4,10,1008,8,1,10,4,10,101,0,8,114,3,8,1002,8,-1,10,101,1,10,10,4,10,108,1,8,10,4,10,101,0,8,135,3,8,1002,8,-1,10,1001,10,1,10,4,10,1008,8,0,10,4,10,102,1,8,158,2,9,9,10,2,2,10,10,3,8,1002,8,-1,10,1001,10,1,10,4,10,1008,8,1,10,4,10,101,0,8,188,1006,0,56,3,8,1002,8,-1,10,1001,10,1,10,4,10,108,1,8,10,4,10,1001,8,0,212,1006,0,76,2,1005,8,10,3,8,102,-1,8,10,1001,10,1,10,4,10,108,1,8,10,4,10,1001,8,0,241,3,8,102,-1,8,10,101,1,10,10,4,10,1008,8,0,10,4,10,1002,8,1,264,1006,0,95,1,1001,12,10,101,1,9,9,1007,9,933,10,1005,10,15,99,109,613,104,0,104,1,21102,838484206484,1,1,21102,1,308,0,1106,0,412,21102,1,937267929116,1,21101,0,319,0,1105,1,412,3,10,104,0,104,1,3,10,104,0,104,0,3,10,104,0,104,1,3,10,104,0,104,1,3,10,104,0,104,0,3,10,104,0,104,1,21102,206312598619,1,1,21102,366,1,0,1105,1,412,21101,179410332867,0,1,21102,377,1,0,1105,1,412,3,10,104,0,104,0,3,10,104,0,104,0,21101,0,709580595968,1,21102,1,400,0,1106,0,412,21102,868389384552,1,1,21101,411,0,0,1106,0,412,99,109,2,21202,-1,1,1,21102,1,40,2,21102,1,443,3,21101,0,433,0,1106,0,476,109,-2,2105,1,0,0,1,0,0,1,109,2,3,10,204,-1,1001,438,439,454,4,0,1001,438,1,438,108,4,438,10,1006,10,470,1102,0,1,438,109,-2,2106,0,0,0,109,4,1202,-1,1,475,1207,-3,0,10,1006,10,493,21102,0,1,-3,21202,-3,1,1,21201,-2,0,2,21101,0,1,3,21102,1,512,0,1106,0,517,109,-4,2105,1,0,109,5,1207,-3,1,10,1006,10,540,2207,-4,-2,10,1006,10,540,22101,0,-4,-4,1106,0,608,21201,-4,0,1,21201,-3,-1,2,21202,-2,2,3,21101,0,559,0,1106,0,517,21201,1,0,-4,21102,1,1,-1,2207,-4,-2,10,1006,10,578,21101,0,0,-1,22202,-2,-1,-2,2107,0,-3,10,1006,10,600,21201,-1,0,1,21102,600,1,0,106,0,475,21202,-2,-1,-2,22201,-4,-2,-4,109,-5,2106,0,0]

In [3]:
class intcode:
    
    def __init__(self, program, input=[], cur_pos=0, rel_base=0, id=0, verbose=False):
        self.memory = program[:]
        self.size = len(program)
        self.input_count = 0
        if isinstance(input, int):
            input = [input]
        self.input = input
        self.cur_pos = cur_pos
        self.relative_base = rel_base
        self.verbose = verbose
        self.id = id
        self.waiting_for_input = False
        self.output = []
        self.logs = []
    
    def _check_mem(self, pos):
        if pos >= self.size:
            self.log(F'Not enough memory - requested {pos}, existing {self.size}', 2)
            self.memory.extend([0] * (pos - self.size + 1))
            self.size = len(self.memory)
            self.log(F'Increased memory to {self.size}', 2)
    
    def _read_mem(self, pointer, mode):
        if mode == 1:
            self.log(F'Immediate mode returning value: {pointer}', 3)
            return pointer
        if mode == 2:
            pointer = pointer + self.relative_base
            self.log(F'Relative mode moving pointer to {pointer}', 3)
        self._check_mem(pointer)
        to_return = self.memory[pointer]
        self.log(F'Returning value {to_return} from position {pointer}', 2)
        return to_return
    
    def _write_mem(self, pointer, mode, value):
        if mode == 2:
            pointer = pointer + self.relative_base
            self.log(F'Relative mode moving pointer to {pointer}', 3)
        self._check_mem(pointer)
        self.memory[pointer] = value
        self.log(F'Writing value {value} to position {pointer}', 2)

        
    def _get_code_and_modes(self, code):
        scode = F'{code:05}'
        return int(scode[-2:]), int(scode[-3]), int(scode[-4]), int(scode[-5])
    
    def run_opcode(self, code, modes):
        if code == 1:
            self.opcode1(modes)
        if code == 2:
            self.opcode2(modes)
        if code == 3:
            self.opcode3(modes)
        if code == 4:
            self.opcode4(modes)
        if code == 5:
            self.opcode5(modes)
        if code == 6:
            self.opcode6(modes)
        if code == 7:
            self.opcode7(modes)
        if code == 8:
            self.opcode8(modes)
        if code == 9:
            self.opcode9(modes)
    
    def opcode1(self, modes):
        mode_a, mode_b, mode_c = modes
        a, b, c = self.memory[self.cur_pos + 1: self.cur_pos + 4]
        self.log(F'Parameters: {a}, {b}, {c}', 1)
        val_a = self._read_mem(a, mode_a)
        val_b = self._read_mem(b, mode_b)
        self._write_mem(c, mode_c, val_a + val_b)
        self.log(F'Wrote {val_a} + {val_b} = {val_a + val_b} to {c}, mode {mode_c}', 2)
        self.cur_pos += 4

    def opcode2(self, modes):
        mode_a, mode_b, mode_c = modes
        a, b, c = self.memory[self.cur_pos + 1: self.cur_pos + 4]
        self.log(F'Parameters: {a}, {b}, {c}', 1)
        val_a = self._read_mem(a, mode_a)
        val_b = self._read_mem(b, mode_b)
        self._write_mem(c, mode_c, val_a * val_b)
        self.log(F'Wrote {val_a} * {val_b} = {val_a * val_b} to {c}, mode {mode_c}', 2)
        self.cur_pos += 4
        
    def opcode3(self, modes):
        mode = modes[0]
        if self.input_count >= len(self.input):
            self.waiting_for_input = True
            self.log(F'Waiting for input, cur_pos: {self.cur_pos}',1)
            return
        param = self.memory[self.cur_pos + 1]
        cur_input = self.input[self.input_count]
        self._write_mem(param, mode, cur_input)
        self.log(F'Wrote {cur_input} to {param}, mode {mode}', 2)
        self.input_count += 1
        self.cur_pos += 2
    
    def opcode4(self, modes):
        mode = modes[0]
        val = self.memory[self.cur_pos + 1]
        self.log(F'Parameter: {val}', 2)
        to_output = self._read_mem(val, mode)
        self.log(F'Writing {to_output} from {val} mode {mode} to output', 2)
        self.output.append(self._read_mem(val, mode))
        self.log(F'New_output: {self.output}', 2)
        self.cur_pos += 2

    def opcode5(self, modes):
        mode_a, mode_b = modes[:2]
        a, b = self.memory[self.cur_pos + 1: self.cur_pos + 3]
        if self._read_mem(a, mode_a) != 0:
            self.cur_pos = self._read_mem(b, mode_b)
        else:
            self.cur_pos += 3
    
    def opcode6(self, modes):
        mode_a, mode_b = modes[:2]
        a, b = self.memory[self.cur_pos + 1: self.cur_pos + 3]
        if self._read_mem(a, mode_a) == 0:
            self.cur_pos = self._read_mem(b, mode_b)
        else:
            self.cur_pos += 3
        
    def opcode7(self, modes):
        mode_a, mode_b, mode_c = modes
        a, b, c = self.memory[self.cur_pos + 1: self.cur_pos + 4]
        val_to_write = 0
        if self._read_mem(a, mode_a) < self._read_mem(b, mode_b):
            val_to_write = 1
        self._write_mem(c, mode_c, val_to_write)
        self.cur_pos += 4
    
    def opcode8(self, modes):
        mode_a, mode_b, mode_c = modes
        a, b, c = self.memory[self.cur_pos + 1: self.cur_pos + 4]
        val_to_write = 0
        if self._read_mem(a, mode_a) == self._read_mem(b, mode_b):
            val_to_write = 1
        self._write_mem(c, mode_c, val_to_write)
        self.cur_pos += 4
        
    def opcode9(self, modes):
        mode = modes[0]
        param = self.memory[self.cur_pos+1]
        val = self._read_mem(param, mode)
        self.relative_base += val
        self.cur_pos += 2
        
    def run(self):
        cur_opcode = 0
        while not self.waiting_for_input and cur_opcode !=99:
            codes = self._get_code_and_modes(self.memory[self.cur_pos])
            cur_opcode = codes[0]
            modes = codes[1:]
            self.log('=====================', 3)
            self.log(F'cur_pos: {self.cur_pos}, opcode: {codes}', 1)
            self.run_opcode(cur_opcode, modes)
        self.log(self.output, 1)
        if cur_opcode == 99:
            return cur_opcode

    def step(self):
        codes = self._get_code_and_modes(self.memory[self.cur_pos])
        self.log('=======================', 3)
        self.log(F'cur_pos: {self.cur_pos}, opcode: {codes}', 1)
        self.run_opcode(codes[0], codes[1:])
    
    def log(self, log_string, level=0):
        self.logs.append(log_string)
        if self.verbose >= level:
            print(log_string)
        
    def add_input(self, input):
        if isinstance(input, list):
            self.input.extend(input)
        elif isinstance(input, int):
            self.input.append(input)
        else:
            raise ValueError('Input must be an int or list')
        self.log(F'Added {input} to input buffer, able to restart', 1)
        self.waiting_for_input = False

In [10]:
panel_painted = {}
cur_x = 0
cur_y = 0
cur_dir = 'u'
dir_map = {
    'u': {'x': 0, 'y': 1},
    'r': {'x': 1, 'y': 0},
    'd': {'x': 0, 'y': -1},
    'l': {'x': -1, 'y': 0}
}


def turn(direction, cur_dir, cur_x, cur_y):
    if direction == 1:
        dirs = ['u','r','d','l','u']
    elif direction == 0:
        dirs = ['u','l','d','r','u']    
    next_dir = dirs[dirs.index(cur_dir) + 1]
    new_x = cur_x + dir_map[next_dir]['x']
    new_y = cur_y + dir_map[next_dir]['y']
    return next_dir, new_x, new_y
    
cur_opcode = None

robot = intcode(robot_code, input=1)
step = 0
print(cur_dir, cur_x, cur_y)
while cur_opcode is None:
    cur_opcode = robot.run()
    color, next_turn = robot.output[-2:]
    panel_painted[(cur_x, cur_y)] = color
    cur_dir, cur_x, cur_y = turn(next_turn, cur_dir, cur_x, cur_y)
#    print(cur_dir, cur_x, cur_y)
    robot.add_input(panel_painted.get((cur_x, cur_y), 0))

    
    

u 0 0


In [16]:
for row in range(6,0,-1):
    y = row - 6
    to_print = ''
    for x in range(43):
        if panel_painted.get((x, y), 0) == 1:
            to_print += 'X'
        else:
            to_print += ' '
    print(to_print)

 X  X XXXX XXX  XXX  XXX  XXXX  XX    XX   
 X  X X    X  X X  X X  X X    X  X    X   
 X  X XXX  X  X X  X X  X XXX  X       X   
 X  X X    XXX  XXX  XXX  X    X XX    X   
 X  X X    X X  X    X X  X    X  X X  X   
  XX  XXXX X  X X    X  X X     XXX  XX    


In [12]:
panel_painted

{(0, -4): 0,
 (0, -3): 0,
 (0, 0): 0,
 (1, -5): 0,
 (1, -4): 1,
 (1, -3): 1,
 (1, -2): 1,
 (1, -1): 1,
 (1, 0): 1,
 (2, -5): 1,
 (2, -4): 0,
 (2, -3): 0,
 (2, -2): 0,
 (2, -1): 0,
 (2, 0): 0,
 (3, -5): 1,
 (3, -4): 0,
 (3, -3): 0,
 (3, -2): 0,
 (3, -1): 0,
 (3, 0): 0,
 (4, -5): 0,
 (4, -4): 1,
 (4, -3): 1,
 (4, -2): 1,
 (4, -1): 1,
 (4, 0): 1,
 (5, -5): 0,
 (5, -4): 0,
 (5, -3): 0,
 (5, -2): 0,
 (5, -1): 0,
 (5, 0): 0,
 (6, -5): 1,
 (6, -4): 1,
 (6, -3): 1,
 (6, -2): 1,
 (6, -1): 1,
 (6, 0): 1,
 (7, -5): 1,
 (7, -4): 0,
 (7, -3): 0,
 (7, -2): 1,
 (7, -1): 0,
 (7, 0): 1,
 (8, -5): 1,
 (8, -4): 0,
 (8, -3): 0,
 (8, -2): 1,
 (8, -1): 0,
 (8, 0): 1,
 (9, -5): 1,
 (9, -4): 0,
 (9, -3): 0,
 (9, -2): 0,
 (9, -1): 0,
 (9, 0): 1,
 (10, -5): 0,
 (10, -4): 0,
 (10, -3): 0,
 (10, -2): 0,
 (10, -1): 0,
 (10, 0): 0,
 (11, -5): 1,
 (11, -4): 1,
 (11, -3): 1,
 (11, -2): 1,
 (11, -1): 1,
 (11, 0): 1,
 (12, -5): 0,
 (12, -4): 0,
 (12, -3): 1,
 (12, -2): 0,
 (12, -1): 0,
 (12, 0): 1,
 (13, -5): 0,
 (13, 

In [13]:
turn_right('l', 2, 5)

('u', 2, 6)

In [45]:
robot = intcode(robot_code)

In [48]:
robot.run()

Waiting for input, cur_pos: 15
[1, 0]


In [47]:
robot.add_input(0)

Added 0 to input buffer, able to restart


In [20]:
robot.input_count

0

In [35]:
robot.step()

IndexError: list index out of range

In [34]:
robot.logs

['=======================',
 'cur_pos: 0, opcode: (3, 0, 0, 0)',
 'Writing value 0 to position 8',
 'Wrote 0 to 8, mode 0',
 '=======================',
 'cur_pos: 2, opcode: (5, 0, 1, 0)',
 'Returning value 0 from position 8',
 '=======================',
 'cur_pos: 5, opcode: (6, 1, 1, 0)',
 'Immediate mode returning value: 0',
 'Immediate mode returning value: 11',
 '=======================',
 'cur_pos: 11, opcode: (4, 1, 0, 0)',
 'Parameter: 1',
 'Immediate mode returning value: 1',
 'Writing 1 from 1 mode 1 to output',
 'Immediate mode returning value: 1',
 'New_output: [1]',
 '=======================',
 'cur_pos: 13, opcode: (4, 1, 0, 0)',
 'Parameter: 0',
 'Immediate mode returning value: 0',
 'Writing 0 from 0 mode 1 to output',
 'Immediate mode returning value: 0',
 'New_output: [1, 0]']

In [36]:
robot.waiting_for_input

False

In [37]:
robot.input_count

1

In [38]:
len(robot.input)

1